In [4]:
# import statements

import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pickle
from math import comb

In [2]:
ibs_graph = pickle.load(open("ibs_graph.pkl", "rb"))

In [23]:
# CDF (cumulative distribution function)
# calculate Pr(v <= x): probability a node v has <= x degree
# Markov's inequality: Pr(v >= x) <= v / E(x), where E is the expected value
# E(x) is computed to be n/2 (assuming edges assigned uniformly and randomly)

n = len(ibs_graph.nodes)
expected = n / 2

degree_dict = {}

cdf = np.zeros(100)
max_deg = 0

for node in ibs_graph.nodes:
    degree = len(ibs_graph.edges(node))
    cdf[degree] += 1
    if degree > max_deg:
        max_deg = degree

cdf_sum = np.zeros(max_deg + 1)
markovs = np.zeros(max_deg + 1)
for i in range(len(cdf_sum)):
    if i > 1:
        cdf_sum[i] = cdf[i] + cdf_sum[i-1]
    else:
        cdf_sum[i] = cdf[i]

    markovs[i] = min(1.0, i / expected)



degrees = np.arange(0, max_deg + 1, 1)

cdf_sum = cdf_sum / n

print(degrees)
print(cdf_sum)
print(markovs)


[0 1 2 3 4 5 6 7]
[0.    0.125 0.125 0.125 0.125 0.125 0.875 1.   ]
[1.   1.   1.   1.   1.   1.25 1.5  1.75]


In [22]:
# plot

plt.figure(figsize=(15, 15), dpi=600)
plt.plot(degrees, cdf_sum, label = "cdf_ibs")
plt.plot(degrees, markovs, label = "Markov's inequality")